# Macro pKa analysis draft for SAMPL7 pKa challenge

In this Jupyter notebook we attempt conversion of microstate transition free energies to macro pKa values for the analysis of the SAMPL7 pKa challenge. We follow the [Selwa et al.](https://doi.org/10.1007/s10822-018-0138-6) SAMPL6 work from JCAMD 2018 in this analysis.

In another notebook, we played with various simple cases and then worked up to the general case. I was able to get the general case to reproduce the macro pKa values for Tielker (ECRISM) and Iorga/Beckstein and the IEFPCM case for several test cases (they provided micro pKa predictions AND macro pKa values) leading me to believe this is now correct.

[Transition networks are available here](https://docs.google.com/presentation/d/16SlgjA3mxwmi6bdbAkhiZrNEc6wJ94hLW8pvTBkI3Uo/edit) as analyzed by Danielle Bergazin

## Let's look at how Nico Tielker does his analysis

This is what Tielker says he  is doing:

$\Delta G_{jB}^{pH} = \Delta G^0_{jB} + \Delta m_{jB}\cdot 1 \cdot (pH-0)$ (Gunner eq. 3)

so 
$N_{j}^{pH} = \frac{10^{-\Delta G^{pH}_{jB}}}{ \sum_i 10^{-\Delta G^{pH}_{iB}}}$ (Gunner eq. 4)

He works out equation 4 for every relative free energy to get $N_j^{pH}$ for every microstate population as a function of pH, which serves as an unknown. 

Then, for each protonation state, sum the populations; consider a protonation state `a` with microstates `j`, `k`, and `l`, then:

$N_{total,a}^{pH} =  \frac{10^{-\Delta G^{pH}_{jB}} + 10^{-\Delta G^{pH}_{kB}}+10^{-\Delta G^{pH}_{lB}}}{ \sum_i 10^{-\Delta G^{pH}_{iB}}}$

Then for two states `a` and `b` which differ only by a single proton, find the pH at which $N_{total,a}^pH = N_{total,b}^{pH}$. He did this in mathematica which has an analytical solver, but a numerical solver could also work. This would be approximate if the pKa values are close together.

If solving for the point where two populations are equal, the partition function in the denominator drops out so we simply need to solve for the point where $N_{total,a}^{pH} = N_{total,b}^{pH}$, or in other words, find pH such that:

$ \sum_{j\ in\ a} 10^{-\Delta G^{pH}_{jB}} - \sum_{j\ in\ b} 10^{-\Delta G^{pH}_{jB}} = 0$

This uses dimensionless units; I was able to reproduce his analysis using a procedure very like this.

## Let's switch to a formalism with units

I like the analysis written out in Selwa et al., JCAMD 32:1203-1216, 2018. This uses a natural logarithm/base e rather than base 10, which is more natural when thinking about free energies, in my view.

In Selwa, the relevant equations are 18 and 19, which give the free energies and populations as a function of pH. I was able to migrate from Tielker's analysis above to this analysis. It's essentially equivalent, we just end up solving for the point where:

$ \sum_{j\ in\ a} exp{(-\beta \Delta G^{pH}_{jB})} - \sum_{j\ in\ b} exp{(-\beta \Delta G^{pH}_{jB})} = 0$

which follows from Selwa equation 18.

## Coding this up

There are several key aspects to coding this up:
- Input format: Here I've selected using a list of state details, consisting of tuples of (state, relative free energy, charge) for all microstates predicted for a given compound
- One critical aspect I'd missed initially is that the free energy of any state depends on pH, so that has to be included
- Solution: Solution will be numerical, as was done in Tielker's analysis
- The numerical solution can on occasion be a bit finicky/not converge well, possibly requiring adjustment of the initial guess or the factor used in the `fsolve` algorithm. I THINK I have settled on values that work well for all cases I tried, but sometimes it may require checking.
- When codified in terms of relative free energies relative to state 0 (formal charge 0), it's important to note that other states with the same formal charge have the same pH dependence, so they will not change in free energy relative to state 0. Care is required at this point, as it's easy to get this wrong.
- To code up the fully general case (which can handle complex cases as well as simple ones) I added code below to skip any charge states which aren't present. So it will output multiple pKa values when all formal charges are present, and only a single one when only two formal charges are present

In [2]:
import numpy as np
from scipy.special import logsumexp
from scipy.optimize import fsolve

# Compute beta and other constants
kB = 1.381*6.02214/1000.0 # [kJ/(mol K)]
beta = 1./(kB*300) # [mol/kJ]
beta = beta*4.186
C_unit = 1/beta*np.log(10)

# Store in list of tupes of (state, relative free energy, charge).


# Test some Tielker/ECRISM cases -- note that these need their units changed before analysis
state_details = [ (3, 7.91*C_unit, -1), (1, -6.66*C_unit, -1), (2, -7.52*C_unit, 0), (4, -12.08*C_unit, 0), (5, -2.33*C_unit, 1)]
# Let's look at molecule 42 as it's a little simpler
#state_details = [ (1, 0.54*C_unit, -1), (2, 0.3*C_unit, 1), (3, -5.05*C_unit, 0)]
# Tielker SM26:
#state_details = [ (1, 5.53*C_unit, -1), (3, 19.92*C_unit, -1), (2, 12.59*C_unit, 0), (4, 4.82*C_unit, 0), (5, 9.95*C_unit, 1)]
# Tielker SM27, to handle simple case:
#state_details = [ (1, 10.17*C_unit, -1)]

#Test some Beckstein/Iorga cases
# Molecule 25
#state_details = [ (1, -2.7, -1), (2, -2, 0), (3, 4.53, -1), (4, -5.75, 0), (5, 0.65, 1)]
# Beckstein/Iorga molecule 42
#state_details = [ (1, 0.15, -1), (2, 0.79, 1), (3, -1.95, 0)]

# Let's try the IEFPCM one for state 42
#state_details = [(1, 6.61, -1), (2, 14.25, 1)] 



# Compute free energy as a function of pH for states
#WITHIN-charge transitions have same pH dependence 
def DeltaG(pH, state):
    for item in state_details:
        if item[0]==state:
            # 0 serves as the reference state; all transitions are away from 0. 
            if item[2]==-1:
                DeltaM=1
            elif item[2]==1:
                DeltaM=-1
            else: DeltaM = 0 # Hack to capture fact that pH dependence of states at same formal charge is same/cancels
            # Compute value
            return (item[1]-pH*DeltaM*C_unit) # Gunner eq 3

# Compute populations for charge states (without normalization, due to laziness/since it'll drop out)
def pop_charge(pH, formal_charge):
    free_energies = []
    for item in state_details:
        if item[2]==formal_charge:
            free_energies.append(-beta*DeltaG(pH,item[0]))
    if formal_charge==0:
        free_energies.append(0*pH)
    return np.exp(logsumexp(free_energies))
    
# Figure out what formal charges are present in states
formal_charges = [ info[2] for info in state_details]
    
# Compute +1 to 0 transition
if 1 in formal_charges:
    init_guess = -5
    func_10 = lambda pH : (pop_charge(pH, 1) - pop_charge(pH, 0))
    pH_solution_1to0 = fsolve(func_10, init_guess, factor = 0.1)
    print("pKa for +1 to 0 transition:", pH_solution_1to0)

# Compute 0 to -1 transition
if -1 in formal_charges:
    init_guess=5
    func_0neg1 = lambda pH : (pop_charge(pH, -1) - pop_charge(pH, 0))
    pH_solution_0toneg1 = fsolve(func_0neg1, init_guess, factor = 0.1)
    print("pKa for 0 to -1 transition:", pH_solution_0toneg1)


pKa for +1 to 0 transition: [-9.75001196]
pKa for 0 to -1 transition: [5.42001196]
